# Big Data - Project

## TODO TOPIC

### Dragan Postolovsky
### Wojciech Taisner

TODO

A 1-page summary of litterature review on studies investigating stackoverflow data

TODO

An enumeration of "interesting" investigations directions for the 10-year dataset of stack overflow (from 09/2008 to 09/2018)

### What were the questions?

TODO

A report with analysis results (including visualisation, and statistics) of the 10 years data. These results are the fruit of the investigations projected earlier

In [ ]:
!ls *.tar.gz

In [47]:
# import dask bag
import dask.bag as db
# import BeatuifulSoup
from bs4 import BeautifulSoup

In [2]:
#files
base = "stackoverflow.com-"

badges = "Badges"
comments = "Comments"
posthistory = "PostHistory"
postlinks = "PostLinks"
posts = "Posts"
tags = "Tags"
users = "Users"
votes = "Votes"

all_files = [badges, comments, postlinks, posts, tags, users, votes] # posthistory, 
# for some reason post history does not want to co-operate
all_files

['Badges', 'Comments', 'PostLinks', 'Posts', 'Tags', 'Users', 'Votes']

# Functions

In [3]:
# non complex parsing (ex badges, tags, etc)
def xml_row_to_tuple(row): # gets data from single xml row (attr values)
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        if v.isdigit():
            v = int(v) #integers take less memory :D
        ret.append(v)
    return tuple(ret);


def xml_headers_to_tuple(row): # gets headers for single xml row (attr names)
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        ret.append(a)
    return tuple(ret);

In [ ]:
# TODO
# read data only form provided attrs
def xml_row_to_tuple_attrs(row, d=tuple("Id")):
    return tuple()

In [ ]:
## select shortest valid header
## if any field from tested header is missing in set of headers, then false
def is_header_valid(setof, element):
    for k in setof:
        for e in element:
            try:
                k.index(e)    
            except ValueError:
                return False
    return True


def find_valid_header(headers):
    # generate list of sorted headers by length ascending
    headers.sort(key=len)
    
    minlen = min(list(map(len, headers)))
    
    # for elements where len(element) == min(len(element)) - other just wont be valid anyway
    # try to find if it is a valid header
    for k in list(filter(lambda x: len(x) == minlen, headers)):
        if is_header_valid(headers, k):
            return k
    return tuple()


def shortest_valid_header(bag):
    # select distinct headers
    headers = bag.map(xml_headers_to_tuple).distinct().compute()
    return find_valid_header(headers)
    
    

In [40]:
# bag summary header + first row
# only bags with proper structure
def summary_bag(bag):
    th = bag.map(xml_headers_to_tuple).take(1)[0]
    for i, v in enumerate(th):
        print('{}: {}'.format(i, v))
    print("")
    tp = bag.map(xml_row_to_tuple).take(1)[0]
    for i, v in enumerate(tp):
        print('{}: {}'.format(i, v))
    print("")

In [38]:
# read bag function
def make_xml_bag(filename):
    extension=".tar.xz"
    compression="xz"
    temp_bag = db.read_text(filename+extension, compression=compression).str.strip()
    temp_bag = temp_bag.filter(lambda x: x.find("row") >= 0)
    return temp_bag

In [ ]:
# check if bag is ok (data are more-a-less structured)
# takes a while of course
def check_bag(bag):
    ma = bag.map(lambda x: len(xml_row_to_tuple(x))).max().compute()
    mi = bag.map(lambda x: len(xml_row_to_tuple(x))).min().compute()
    if ma == mi:
        return True
    print(ma, mi)
    return False

## Review of files structure (probably to be removed later)

In [41]:
# refiew structure of all files
# takes a second
for filename in all_files:
    print(filename)
    temp_bag = make_xml_bag(filename)
    summary_bag(temp_bag)

Badges
0: Id
1: UserId
2: Name
3: Date
4: Class
5: TagBased

0: 82946
1: 3718
2: Teacher
3: 2008-09-15T08:55:03.923
4: 3
5: False

Comments
0: Id
1: PostId
2: Score
3: Text
4: CreationDate
5: UserId

0: 2
1: 35314
2: 8
3: Yeah, I didn't believe it until I created a console app - but good lord!  Why would they give you the rope to hang yourself!  I hated that about VB.NET - the OrElse and AndAlso keywords!
4: 2008-09-06T08:09:52.330
5: 3

PostLinks
0: Id
1: CreationDate
2: PostId
3: RelatedPostId
4: LinkTypeId

0: 19
1: 2010-04-26T02:59:48.130
2: 109
3: 32412
4: 1

Posts
0: Id
1: PostTypeId
2: AcceptedAnswerId
3: CreationDate
4: Score
5: ViewCount
6: Body
7: OwnerUserId
8: LastEditorUserId
9: LastEditorDisplayName
10: LastEditDate
11: LastActivityDate
12: Title
13: Tags
14: AnswerCount
15: CommentCount
16: FavoriteCount

0: 6
1: 1
2: 31
3: 2008-07-31T22:08:08.620
4: 261
5: 16799
6: <p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relativ

# Some real stuff

## Tags

In [7]:
# read tags bag, select rows only
tags_bag = db.read_text(tags+".tar.xz", compression="xz").str.strip()
tags_bag = tags_bag.filter(lambda x: x.find("row") >= 0)

### Simple tests [Polygon]

In [8]:
for row in tags_bag.take(4):
    print(row);
    print(xml_row_to_tuple(row))
    #soup = BeautifulSoup(row, "xml")
    #print(soup.row['Id'], soup.row['TagName'], soup.row['Count'])

<row Id="1" TagName=".net" Count="276857" ExcerptPostId="3624959" WikiPostId="3607476" />
(1, '.net', 276857, 3624959, 3607476)
<row Id="2" TagName="html" Count="789778" ExcerptPostId="3673183" WikiPostId="3673182" />
(2, 'html', 789778, 3673183, 3673182)
<row Id="3" TagName="javascript" Count="1723731" ExcerptPostId="3624960" WikiPostId="3607052" />
(3, 'javascript', 1723731, 3624960, 3607052)
<row Id="4" TagName="css" Count="563816" ExcerptPostId="3644670" WikiPostId="3644669" />
(4, 'css', 563816, 3644670, 3644669)


In [ ]:
test = tags_bag.map(xml_row_to_tuple).take(1)
test[0]

In [10]:
header = tags_bag.map(xml_headers_to_tuple).take(1)[0]
header

('Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId')

In [11]:
print(type(tags_bag))
print(type(test))
#tags_bag.map(xml_row_to_tuple).take(1)

<class 'dask.bag.core.Bag'>
<class 'list'>


In [12]:
# takes a while
check_bag(tags_bag)

5 3


False

In [13]:
tags_bag.map(xml_row_to_tuple).filter(lambda x: len(x)<5).count().compute()

14218

In [14]:
tags_bag.map(xml_headers_to_tuple).filter(lambda x: len(x)==3).distinct().compute()
#tags_bag.count().compute()
#
# 14k / 55k has only 3/5 tags (two last missing), need to be dropped
#

[('Id', 'TagName', 'Count')]

In [22]:
%%time
dt = tags_bag.map(xml_headers_to_tuple).distinct().compute()
print(dt)

[('Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId'),
 ('Id', 'TagName', 'Count')]

In [35]:
%%time
if len(dt) > 1:
    header = shortest_valid_header(dt)
    print(header)

('Id', 'TagName', 'Count')
CPU times: user 23.6 ms, sys: 27.4 ms, total: 51 ms
Wall time: 8.06 s


## Badges

In [42]:
badges_bag = make_xml_bag(badges)

In [44]:
dba = badges_bag.map(xml_headers_to_tuple).distinct().compute()
dba

[('Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased')]

In [45]:
%%time
## only one distinct header, do not need to compute this
if len(dba) > 1:
    b_header = find_valid_header(dba)
    print(b_header)

('Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased')
CPU times: user 2.16 s, sys: 833 ms, total: 2.99 s
Wall time: 1h 15min 40s


## Comments

In [ ]:
comments_bag = make_xml_bag(comments)

In [ ]:
%%time
dcb = comments_bag.map(xml_headers_to_tuple).distinct().compute()
print(dcb)

In [ ]:
%%time
## only one distinct header, do not need to compute this
if len(dcb) > 1:
    b_header = find_valid_header(dcb)
    print(b_header)

## Post Links

In [ ]:
postlinks_bag = make_xml_bag(postlinks)

In [ ]:
%%time
dplb = dostlinks_bag.map(xml_headers_to_tuple).distinct().compute()
print(dplb)

In [ ]:
%%time
## only one distinct header, do not need to compute this
if len(dplb) > 1:
    b_header = find_valid_header(dplb)
    print(b_header)

## Users

In [ ]:
users_bag = make_xml_bag(users)

In [ ]:
%%time
dub = users_bag.map(xml_headers_to_tuple).distinct().compute()
print(dub)

In [ ]:
%%time
## only one distinct header, do not need to compute this
if len(dub) > 1:
    b_header = find_valid_header(dub)
    print(b_header)

## Votes

In [48]:
votes_bag = make_xml_bag(votes)

In [ ]:
%%time
dv = votes_bag.map(xml_headers_to_tuple).distinct().compute()
print(dv)

In [ ]:
%%time
## only one distinct header, do not need to compute this
if len(dv) > 1:
    b_header = find_valid_header(dv)
    print(b_header)

## Posts History
seems that data are too big and too complicated to process them right now

## Posts and Answers 

In [15]:
## split posts to posts and answers
# takes a while
# quite a long ...
# man im gonna take a shower
%%time
both_bag = db.read_text(posts+".tar.xz", compression="xz").str.strip()
both_bag = both_bag.filter(lambda x: x.find('row') >=0 )
totc = both_bag.count().compute()
totc # total count

42850538

In [16]:
%%time
answers_bag = both_bag.filter(lambda x: x.find("PostTypeId=\"2\"") >= 0)
#answers_bag.map(xml_row_to_tuple).take(1)
ansc = answers_bag.count().compute()
ansc # answers count

25908201

In [17]:
%%time
posts_bag = both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") >= 0)
#posts_bag.map(xml_row_to_tuple).take(1)
posc = posts_bag.count().compute()
posc # posts count

16845942

In [18]:
missing = totc - ansc - posc # how many missing
missing

96395

In [23]:
# try to find missing vol 2 works
both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") < 0 and x.find("PostTypeId=\"2\"") < 0).count().compute()

96395

In [28]:
# check whats wrong with data -> undefined types of posts
both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") < 0 and x.find("PostTypeId=\"2\"") < 0).take(1)

('<row Id="3606997" PostTypeId="5" CreationDate="2010-07-22T13:36:17.203" Score="0" Body="&lt;h2&gt;What is C++?&lt;/h2&gt;&#xA;&#xA;&lt;p&gt;C++ is a &lt;a href=&quot;https://en.wikipedia.org/wiki/Type_system#Static_typing&quot; rel=&quot;nofollow noreferrer&quot;&gt;statically-typed&lt;/a&gt;, free-form, (usually) compiled, multi-paradigm, intermediate-level general-purpose programming language; not to be confused with &lt;a href=&quot;https://stackoverflow.com/tags/c/info&quot;&gt;C&lt;/a&gt; or &lt;a href=&quot;https://stackoverflow.com/tags/c%2b%2b-cli/info&quot;&gt;C++/CLI&lt;/a&gt;. It was developed in the early 1980s by &lt;a href=&quot;https://www.stroustrup.com/&quot; rel=&quot;nofollow noreferrer&quot;&gt;Bjarne Stroustrup&lt;/a&gt;, initially as a set of extensions to the C programming language. Building on C, &lt;a href=&quot;https://en.wikipedia.org/wiki/C++&quot; rel=&quot;nofollow noreferrer&quot;&gt;C++&lt;/a&gt; improved &lt;a href=&quot;https://en.wikipedia.org/wiki/

In [37]:
#%%time
## check data structure across answers
## dont run it, takes a while
#da = answers_bag.map(xml_headers_to_tuple).distinct().compute()
#da

In [36]:
#%%time
## check data structure across posts
## dont run it, takes a while
#dp = posts_bag.map(xml_headers_to_tuple).distinct().compute()
#dp